In [1]:
import numpy as np 
import pandas as pd 
from mlxtend.frequent_patterns import apriori

In [2]:
!pip install mlxtend

You should consider upgrading via the 'c:\python\python.exe -m pip install --upgrade pip' command.


In [3]:
def createC1(dataSet):
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
                
    
    return list(map(frozenset, C1))
    
    

In [4]:
def scanD(D, Ck, minSupport):
    ssCnt = {}
    for tid in D:
        for can in Ck:
            if can.issubset(tid):
                if not can in ssCnt: ssCnt[can]=1
                else: ssCnt[can] += 1
    numItems = float(len(D))
    retList = []
    supportData = {}
    for key in ssCnt:
        support = ssCnt[key]/numItems
        if support >= minSupport:
            retList.insert(0,key)
        supportData[key] = support
    return retList, supportData

In [65]:
df = pd.read_csv('data.csv')
df = df.dropna()
df.head(20)


,T1,T2,T3
0,A,B,C
1,A,C,Nan
2,A,D,Nan
3,B,E,F


In [6]:
dataSet = df.values.tolist()
dataSet

[['A', 'B', 'C'], ['A', 'C', 'Nan'], ['A', 'D', 'Nan'], ['B', 'E', 'F']]

In [7]:
C1 = createC1(dataSet)

In [8]:
C1

[frozenset({'A'}),
 frozenset({'B'}),
 frozenset({'C'}),
 frozenset({'Nan'}),
 frozenset({'D'}),
 frozenset({'E'}),
 frozenset({'F'})]

In [9]:
#D is a dataset in the setform.

D = list(map(set,dataSet))

In [10]:
D

[{'A', 'B', 'C'}, {'A', 'C', 'Nan'}, {'A', 'D', 'Nan'}, {'B', 'E', 'F'}]

In [11]:
L1,suppData = scanD(D,C1,0.5)
L1

[frozenset({'Nan'}), frozenset({'C'}), frozenset({'B'}), frozenset({'A'})]

In [12]:
def aprioriGen(Lk, k): #creates Ck
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1, lenLk): 
            L1 = list(Lk[i])[:k-2]; L2 = list(Lk[j])[:k-2]
            L1.sort(); L2.sort()
            if L1==L2: #if first k-2 elements are equal
                retList.append(Lk[i] | Lk[j]) #set union
    return retList

In [13]:
def apriori(dataSet, minSupport = 0.5):
    C1 = createC1(dataSet)
    D = list(map(set, dataSet))
    L1, supportData = scanD(D, C1, minSupport)
    L = [L1]
    k = 2
    while (len(L[k-2]) > 0):
        Ck = aprioriGen(L[k-2], k)
        Lk, supK = scanD(D, Ck, minSupport)#scan DB to get Lk
        supportData.update(supK)
        L.append(Lk)
        k += 1
    return L, supportData

In [14]:
L,suppData = apriori(dataSet)

In [15]:
L

[[frozenset({'Nan'}), frozenset({'C'}), frozenset({'B'}), frozenset({'A'})],
 [frozenset({'A', 'Nan'}), frozenset({'A', 'C'})],
 []]

In [16]:
L[0]

[frozenset({'Nan'}), frozenset({'C'}), frozenset({'B'}), frozenset({'A'})]

In [17]:
L[1]

[frozenset({'A', 'Nan'}), frozenset({'A', 'C'})]

In [18]:
L[2]

[]

In [19]:
L[3]

IndexError: list index out of range

In [20]:
aprioriGen(L[0],2)

[frozenset({'C', 'Nan'}),
 frozenset({'B', 'Nan'}),
 frozenset({'A', 'Nan'}),
 frozenset({'B', 'C'}),
 frozenset({'A', 'C'}),
 frozenset({'A', 'B'})]

In [21]:
def generateRules(L, supportData, minConf=0.7):  #supportData is a dict coming from scanD
    bigRuleList = []
    for i in range(1, len(L)):#only get the sets with two or more items
        for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet]
            if (i > 1):
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else:
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)
    return bigRuleList         

In [22]:
def calcConf(freqSet, H, supportData, brl, minConf=0.7):
    prunedH = [] #create new list to return
    for conseq in H:
        conf = supportData[freqSet]/supportData[freqSet-conseq] #calc confidence
        if conf >= minConf: 
            print (freqSet-conseq,'-->',conseq,'conf:',conf)
            brl.append((freqSet-conseq, conseq, conf))
            prunedH.append(conseq)
    return prunedH

In [23]:
def rulesFromConseq(freqSet, H, supportData, brl, minConf=0.7):
    m = len(H[0])
    if (len(freqSet) > (m + 1)): #try further merging
        Hmp1 = aprioriGen(H, m+1)#create Hm+1 new candidates
        Hmp1 = calcConf(freqSet, Hmp1, supportData, brl, minConf)
        if (len(Hmp1) > 1):    #need at least two sets to merge
            rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf)

In [24]:
L,suppData= apriori(dataSet,minSupport=0.5)

In [25]:
rules= generateRules(L,suppData, minConf=0.7)

frozenset({'Nan'}) --> frozenset({'A'}) conf: 1.0
frozenset({'C'}) --> frozenset({'A'}) conf: 1.0


In [26]:
datafile = pd.read_csv('data.csv')

In [27]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_ary = te.fit(datafile).transform(datafile)
df = pd.DataFrame(te_ary, columns=te.columns_)
df


,1,2,3,T
0,True,False,False,True
1,False,True,False,True
2,False,False,True,True
3,False,False,False,False


In [28]:
datafile = pd.read_csv('data.csv')

In [29]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_ary = te.fit(datafile).transform(datafile)
df = pd.DataFrame(te_ary, columns=te.columns_)
df

,1,2,3,T
0,True,False,False,True
1,False,True,False,True
2,False,False,True,True
3,False,False,False,False


In [30]:
from mlxtend.frequent_patterns import apriori

apriori(df, min_support=0.6)

,support,itemsets
0,0.75,(3)


In [31]:
apriori(df, min_support=0.5, use_colnames=True)

,support,itemsets
0,0.75,(T)


In [32]:
df = pd.read_csv('lastfm.csv', header=None)
df = df.dropna()


c:\python\lib\site-packages\IPython\core\interactiveshell.py:3155: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [49]:
df = df.head()
df

,0,1,2,3
0,user,artist,sex,country
1,1,red hot chili peppers,f,Germany
2,1,the black dahlia murder,f,Germany
3,1,goldfrapp,f,Germany
4,1,dropkick murphys,f,Germany


In [34]:
df = df.dropna()
dataSet = df.values.tolist()
dataSet

[['user', 'artist', 'sex', 'country'],
 ['1', 'red hot chili peppers', 'f', 'Germany'],
 ['1', 'the black dahlia murder', 'f', 'Germany'],
 ['1', 'goldfrapp', 'f', 'Germany'],
 ['1', 'dropkick murphys', 'f', 'Germany']]

In [35]:
C1 = createC1(dataSet)


In [36]:
D = list(map(set,dataSet))

In [37]:
L1,suppData = scanD(D,C1,0.5)
L1

[frozenset({'Germany'}), frozenset({'f'}), frozenset({'1'})]

In [38]:
def aprioriGen(Lk, k): #creates Ck
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1, lenLk): 
            L1 = list(Lk[i])[:k-2]; L2 = list(Lk[j])[:k-2]
            L1.sort(); L2.sort()
            if L1==L2: #if first k-2 elements are equal
                retList.append(Lk[i] | Lk[j]) #set union
    return retList

In [39]:
def apriori(dataSet, minSupport = 0.5):
    C1 = createC1(dataSet)
    D = list(map(set, dataSet))
    L1, supportData = scanD(D, C1, minSupport)
    L = [L1]
    k = 2
    while (len(L[k-2]) > 0):
        Ck = aprioriGen(L[k-2], k)
        Lk, supK = scanD(D, Ck, minSupport)#scan DB to get Lk
        supportData.update(supK)
        L.append(Lk)
        k += 1
    return L, supportData

In [40]:
L,suppData = apriori(dataSet)

In [41]:
L

[[frozenset({'Germany'}), frozenset({'f'}), frozenset({'1'})],
 [frozenset({'1', 'f'}),
  frozenset({'1', 'Germany'}),
  frozenset({'Germany', 'f'})],
 [frozenset({'1', 'Germany', 'f'})],
 []]

In [42]:
L[0]

[frozenset({'Germany'}), frozenset({'f'}), frozenset({'1'})]

In [43]:
L[1]

[frozenset({'1', 'f'}),
 frozenset({'1', 'Germany'}),
 frozenset({'Germany', 'f'})]

In [44]:
L[2]

[frozenset({'1', 'Germany', 'f'})]

In [45]:
L[3]

[]

In [46]:
aprioriGen(L[0],2)

[frozenset({'Germany', 'f'}),
 frozenset({'1', 'Germany'}),
 frozenset({'1', 'f'})]

In [47]:
L,suppData= apriori(dataSet,minSupport=0.5)

In [48]:
rules= generateRules(L,suppData, minConf=0.7)

frozenset({'f'}) --> frozenset({'1'}) conf: 1.0
frozenset({'1'}) --> frozenset({'f'}) conf: 1.0
frozenset({'Germany'}) --> frozenset({'1'}) conf: 1.0
frozenset({'1'}) --> frozenset({'Germany'}) conf: 1.0
frozenset({'Germany'}) --> frozenset({'f'}) conf: 1.0
frozenset({'f'}) --> frozenset({'Germany'}) conf: 1.0
frozenset({'Germany'}) --> frozenset({'1', 'f'}) conf: 1.0
frozenset({'f'}) --> frozenset({'1', 'Germany'}) conf: 1.0
frozenset({'1'}) --> frozenset({'f', 'Germany'}) conf: 1.0


In [53]:
df = pd.read_csv('lastfm.csv', header=None)
df = df.dropna()
df = df.tail(20)
df

c:\python\lib\site-packages\IPython\core\interactiveshell.py:3155: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,0,1,2,3
289936,19718,sufjan stevens,f,Canada
289937,19718,crystal castles,f,Canada
289938,19718,regina spektor,f,Canada
289939,19718,the velvet underground,f,Canada
289940,19718,joy division,f,Canada
289941,19718,the distillers,f,Canada
289942,19718,metric,f,Canada
289943,19718,peter bjorn and john,f,Canada
289944,19718,ramones,f,Canada
289945,19718,the kinks,f,Canada


In [54]:
dataSet = df.values.tolist()
dataSet

[[19718, 'sufjan stevens', 'f', 'Canada'],
 [19718, 'crystal castles', 'f', 'Canada'],
 [19718, 'regina spektor', 'f', 'Canada'],
 [19718, 'the velvet underground', 'f', 'Canada'],
 [19718, 'joy division', 'f', 'Canada'],
 [19718, 'the distillers', 'f', 'Canada'],
 [19718, 'metric', 'f', 'Canada'],
 [19718, 'peter bjorn and john', 'f', 'Canada'],
 [19718, 'ramones', 'f', 'Canada'],
 [19718, 'the kinks', 'f', 'Canada'],
 [19718, 'david bowie', 'f', 'Canada'],
 [19718, 'mgmt', 'f', 'Canada'],
 [19718, 'cold war kids', 'f', 'Canada'],
 [19718, 'fleetwood mac', 'f', 'Canada'],
 [19718, 'arcade fire', 'f', 'Canada'],
 [19718, 'bob dylan', 'f', 'Canada'],
 [19718, 'pixies', 'f', 'Canada'],
 [19718, 'the clash', 'f', 'Canada'],
 [19718, 'a tribe called quest', 'f', 'Canada'],
 [19718, 'radiohead', 'f', 'Canada']]

In [56]:
C1 = createC1(dataSet)
D = list(map(set,dataSet))
L1,suppData = scanD(D,C1,0.5)
L1
def aprioriGen(Lk, k): #creates Ck
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1, lenLk): 
            L1 = list(Lk[i])[:k-2]; L2 = list(Lk[j])[:k-2]
            L1.sort(); L2.sort()
            if L1==L2: #if first k-2 elements are equal
                retList.append(Lk[i] | Lk[j]) #set union
    return retList
def apriori(dataSet, minSupport = 0.5):
    C1 = createC1(dataSet)
    D = list(map(set, dataSet))
    L1, supportData = scanD(D, C1, minSupport)
    L = [L1]
    k = 2
    while (len(L[k-2]) > 0):
        Ck = aprioriGen(L[k-2], k)
        Lk, supK = scanD(D, Ck, minSupport)#scan DB to get Lk
        supportData.update(supK)
        L.append(Lk)
        k += 1
    return L, supportData
L,suppData = apriori(dataSet)
L
L[0]
L[1]
L[2]
L[3]
aprioriGen(L[0],2)
L,suppData= apriori(dataSet,minSupport=0.5)
rules= generateRules(L,suppData, minConf=0.7)




frozenset({19718}) --> frozenset({'f'}) conf: 1.0
frozenset({'f'}) --> frozenset({19718}) conf: 1.0
frozenset({19718}) --> frozenset({'Canada'}) conf: 1.0
frozenset({'Canada'}) --> frozenset({19718}) conf: 1.0
frozenset({'f'}) --> frozenset({'Canada'}) conf: 1.0
frozenset({'Canada'}) --> frozenset({'f'}) conf: 1.0
frozenset({19718}) --> frozenset({'Canada', 'f'}) conf: 1.0
frozenset({'f'}) --> frozenset({'Canada', 19718}) conf: 1.0
frozenset({'Canada'}) --> frozenset({'f', 19718}) conf: 1.0


In [59]:
L


[[frozenset({'Canada'}), frozenset({'f'}), frozenset({19718})],
 [frozenset({19718, 'f'}),
  frozenset({19718, 'Canada'}),
  frozenset({'Canada', 'f'})],
 [frozenset({19718, 'Canada', 'f'})],
 []]

In [60]:
L[0]

[frozenset({'Canada'}), frozenset({'f'}), frozenset({19718})]

In [61]:
L[1]

[frozenset({19718, 'f'}),
 frozenset({19718, 'Canada'}),
 frozenset({'Canada', 'f'})]

In [62]:
L[2]

[frozenset({19718, 'Canada', 'f'})]

In [63]:
L[3]

[]

From the dataset lastfm.csv, we take two other subsets, by using loc[] function:
dataset4=df.loc[40:60]
dataset5=df.loc[60:80]
We have to now run the code from the lines 34 to 48